In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [ ]:
test_set = torchvision.datasets.FashionMNIST(
    root='./data'
    ,train=False
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [ ]:
remote_loader = torch.utils.data.DataLoader(
    test_set, batch_size=1, shuffle=True)

In [ ]:
batch = next(iter(remote_loader))
images, labels = batch

In [ ]:
infer_url = 'https://changeme/v2/models/imageclassifier/infer'
bearer = 'Bearer change me'

In [ ]:
import requests

def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "input.1",
                "shape": [1, 1, 28, 28],
                "datatype": "FP32",
                "data": data
            }
        ]
    }
    
    headers = {'Authorization': bearer}

    response = requests.post(infer_url, json=json_data, verify=False, headers=headers)
    response_dict = response.json()
    return response_dict['outputs'][0]['data']

In [ ]:
images[0].unsqueeze(0).shape

In [ ]:
prediction = rest_request(images[0].unsqueeze(0).tolist())
output = torch.tensor(prediction)

print("Prediction:" + str(output.argmax().item()))
print("Actual:" + str(labels[0].item()))

In [ ]:
plt.figure(figsize=(28,28))
batch = next(iter(remote_loader))
images, labels = batch
prediction = rest_request(images[0].unsqueeze(0).tolist())
pred = torch.tensor(prediction)
output = pred.argmax(dim=0)


image = images[0]
label = labels[0]               
ax = plt.subplot(1,10,1)
plt.imshow(image.reshape(28,28), cmap='gray')
plt.axis('off')
plt.title(test_set.classes[label.item()], fontsize=28)
plt.text(0.5,-0.3,test_set.classes[output.item()], fontsize=28,color='red',ha='center',transform=ax.transAxes)
plt.show()